In [ ]:
"""
1. Using the data_sample data set provided, put the airline carriers in order from most used to least used.
2. Which visa type category has the most and least arrivals to the U.S.?
3.Using the cities-us dataset, which state has the most cities?
"""

In [1]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.appName("cities_visas").getOrCreate()

23/05/15 15:55:46 WARN Utils: Your hostname, Ravis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.6.23 instead (on interface en0)
23/05/15 15:55:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/15 15:55:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/15 15:55:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
airlines_df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .option("ignoreLeadingWhiteSpace", True) \
            .load("data_sample.csv")


In [6]:
cities_df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load("cities-us.csv")

In [10]:
def removeDuplicates(df):
    dup_count = df.count()
    no_dup_count = df.distinct().count()
    diff = dup_count - no_dup_count
    if diff:
        print(f"dataframe has duplicates, count of duplicates is {diff}. Dropping these duplicates")
        return df.dropDuplicates()
    else:
        print("no duplicates")
        return df

In [11]:
# drop unnamed columns in the dataset
airlines_df = airlines_df.drop(col("_c0"))

In [12]:
# drop duplicates from the dataset
airlines_df = removeDuplicates(airlines_df)

23/05/15 16:05:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


no duplicates


In [14]:
# Airline Carriers in order from most used to least
airlineUsage = airlines_df.groupBy("airline").agg(count("airline").alias("count"))

In [15]:
airlineUsage.orderBy(desc("count")).show(airlineUsage.count())

+-------+-----+
|airline|count|
+-------+-----+
|     AA|  109|
|     UA|   84|
|     DL|   77|
|     BA|   53|
|     LH|   45|
|     VS|   43|
|     JL|   24|
|     KE|   21|
|     AF|   21|
|     KL|   19|
|     EK|   19|
|     Y4|   19|
|     SK|   18|
|     AV|   17|
|     QF|   17|
|     AM|   16|
|     QR|   15|
|     NZ|   12|
|     OZ|   12|
|     NH|   11|
|     DY|   11|
|     CM|   11|
|     HA|   11|
|     EI|   11|
|     TK|   11|
|     LA|   10|
|     SU|   10|
|     LX|   10|
|     BR|   10|
|     MU|   10|
|     AR|    9|
|     AD|    8|
|     AB|    8|
|     IB|    7|
|     JJ|    7|
|     B6|    7|
|     CX|    7|
|     7C|    7|
|     CA|    7|
|     BW|    5|
|    VES|    5|
|     AI|    5|
|     WN|    5|
|     CI|    4|
|     NK|    4|
|     AY|    4|
|     EQ|    4|
|     LJ|    4|
|     SE|    4|
|    *GA|    4|
|     JQ|    4|
|     SQ|    4|
|     FI|    3|
|     AZ|    3|
|     4O|    3|
|     EV|    3|
|     SV|    3|
|     3U|    3|
|     MT|    3|
|     S3

In [17]:
# visa with most and least arrivals to USA
airlines_df.groupBy("visatype").agg(count("cicid").alias("arrivalCount")).orderBy(desc("arrivalCount")).show()

+--------+------------+
|visatype|arrivalCount|
+--------+------------+
|      WT|         443|
|      B2|         356|
|      WB|          91|
|      B1|          61|
|     GMT|          27|
|      F1|          10|
|      CP|           5|
|      F2|           3|
|      E2|           3|
|      M1|           1|
+--------+------------+



In [31]:
# remove footer
cities_df = cities_df.limit(cities_df.count() - 1)

In [32]:
cities_df = removeDuplicates(cities_df)

no duplicates


In [33]:
cities_df = cities_df.withColumnRenamed("SR.No", "SerialNum")

In [34]:
# Drop columns with null in primary key
cities = cities_df.na.drop(subset=["SerialNum"])

In [35]:
cities.show()

+---------+-------------+--------------+-------------+-----------+--------------+-----------------------+------------------+
|SerialNum|         City|         State|2012 estimate|2010 Census|2010 land area|2010 population density|          Location|
+---------+-------------+--------------+-------------+-----------+--------------+-----------------------+------------------+
|        1|     New York|      New York|    83,36,697|  81,75,133| 302.643�sq�mi|       27,012�per�sq�mi| 40.6643N 73.9385W|
|        2|  Los Angeles|    California|    38,57,799|  37,92,621| 468.670�sq�mi|        8,092�per�sq�mi|34.0194N 118.4108W|
|        3|      Chicago|      Illinois|    27,14,856|  26,95,598| 227.635�sq�mi|       11,842�per�sq�mi| 41.8376N 87.6818W|
|        4|      Houston|         Texas|    21,60,821|  21,00,263| 599.589�sq�mi|        3,501�per�sq�mi| 29.7805N 95.3863W|
|        5| Philadelphia|  Pennsylvania|    15,47,607|  15,26,006| 134.101�sq�mi|       11,379�per�sq�mi| 40.0094N 75.1333W|


In [38]:
grouped_df = cities.groupBy("State").agg(count("City").alias("count"))
grouped_df.show()

+--------------------+-----+
|               State|count|
+--------------------+-----+
|            New York|    5|
|          California|   69|
|            Illinois|    8|
|               Texas|   32|
|        Pennsylvania|    4|
|             Arizona|   10|
|             Florida|   20|
|             Indiana|    4|
|                Ohio|    6|
|      North Carolina|    9|
|            Michigan|    7|
|           Tennessee|    6|
|       Massachusetts|    5|
|          Washington|    7|
|            Colorado|   11|
|District of Columbia|    1|
|            Maryland|    1|
|            Kentucky|    2|
|              Oregon|    4|
|            Oklahoma|    4|
+--------------------+-----+
only showing top 20 rows



In [39]:
print(f"***** Highest number of cities *****")
grouped_df.orderBy(desc(col("count"))).limit(1).show()

***** Highest number of cities *****
+----------+-----+
|     State|count|
+----------+-----+
|California|   69|
+----------+-----+

